### Libraries

In [161]:
import math
import numpy as np

import pandas 
import plotly.express as px

### Constants

In [162]:
# @title Gauss-Laguerre constants
# Notation: (abscissa, weight)
GAUSS_LAGUERRE = [
    (0.222847, 0.458964),
    (1.188932, 0.417),
    (2.992736, 0.113373),
    (5.775144, 0.0103992),
    (9.837467, 0.000261017),
    (15.982874, 0.000000898548)
]

### Functions

In [163]:
# @title Weibull distribution function
def f_Weibull (t, k, mu):
    varLambdaDenom = lambda s, k : math.pow (s, 1/k)
    
    area = 0

    for i in range(6):
        area += GAUSS_LAGUERRE[i][1] * varLambdaDenom(GAUSS_LAGUERRE[i][0], k)
    
    varLambda = mu / area
    
    # print("---> lambda=", varLambda, "k=", k, ", t=", t)
    # print("---> (t/lambda)^(k-1) =", (t/varLambda) ** (k-1))
    # print("---> e^-(t/lambda)^-k =", (math.e ** ((t/varLambda) ** k)))
    f = (k/varLambda) * ((t/varLambda) ** k-1) * (math.e ** ((t/varLambda) ** k))

    return f

In [164]:
# @title Integral of Weibull function
def cdf_Weibull (t, k, mu):
    area = 0
    
    for i in range(6):
        area += GAUSS_LAGUERRE[i][1] * f_Weibull( -GAUSS_LAGUERRE[i][0] + t, k, mu)

    return area

In [165]:
# Inverse survivability
# solved as root-finding problem using bisection method
def surv_inv (alpha, a, b, k, mu):
    isDiffSign = lambda x1, x2 : np.sign(x1) != np.sign(x2)
    f = lambda t : cdf_Weibull(t, k, mu) - alpha

    tol = 1.0e-4

    fa = f(a)
    if fa == 0.0:
        return a
    
    fb = f(b)
    if fb == 0.0:
        return b
    
    if not isDiffSign(fa, fb):
        return None
    
    n = int (math.ceil (math.log(abs(b-a) / tol) / math.log(2.0)))

    for i in range(n):
        c = 0.5 * (a + b)
        fc = f(c)

        if (abs(fc) > abs(fa)) and (abs(fc) > abs(fb)):
            return None

        if fc == 0.0:
            return c

        if isDiffSign(fa, fc):
            a = c
            fa = fc
        
        elif isDiffSign(fb, fc):
            b = c
            fb = fc

    return 0.5 * (a+b)

### Evaluation

In [166]:
# Average life expectancy
expected_value = 78

# List for values
y_t_decr = []
y_t_cons = []
y_t_incr = []
x_values = []

In [167]:
# For alpha in (0,1) with 0.01 step
for alpha in range(1, 100, 1):
    alpha /= 100

    t_decr = surv_inv (alpha, 0, 100, 0.5, expected_value)
    t_cons = surv_inv (alpha, 0, 100, 1, expected_value)
    t_incr = surv_inv (alpha, 0, 100, 2, expected_value)

    x_values.append(alpha)

    y_t_decr.append(t_decr)
    y_t_cons.append(t_cons)
    y_t_incr.append(t_incr)

### Plotting